In [1]:
# ruff: noqa
import sys, os
sys.path.append(os.path.abspath("./../feedback-grape"))
sys.path.append(os.path.abspath("./../"))

# ruff: noqa
from feedback_grape.fgrape import optimize_pulse, evaluate_on_longer_time
from helpers import (
    init_fgrape_protocol,
    test_implementations,
    generate_random_discrete_state,
    generate_random_bloch_state,
    generate_excited_state,
    generate_ground_state,
)
from tqdm import tqdm
import jax
import jax.numpy as jnp
import numpy as np
from library.utils.FgResult_to_dict import FgResult_to_dict
import json

test_implementations()

In [ ]:
# Training and evaluation parameters
dir_ = "./01_results"

training_params = {
    "N_samples": 5, # Number of random initial states to sample
    "samples_offset": 0, # Offset for sample indexing
    "N_training_iterations": 10000, # Number of training iterations
    "learning_rate": 0.01, # Learning rate
    "convergence_threshold": 1e-6,
    "batch_size": 16,
    "eval_batch_size": 16,
    "evaluation_time_steps": 100,
}
N_parallel_threads = 64 # Number of parallel threads for training

# Physical Parameters to test

#num_time_steps : Number of time steps in the control pulse
#lut_memory : Amount of measurements the LUT remembers (depth of LUT)
#reward_weights : Weights for the reward at each time step. Default only weights last timestep [0, 0, ... 0, 1]
#N_qubits : Number of parallel chains to simulate

experiments = [ # (timesteps, lut_memory, reward_weights, N_qubits, N_meas, gamma_p, gamma_m, training_state_type, evaluation_state_type)
    (3, 2, [1,1,1], 3, 2, gamma_p, gamma_m, "bloch", "bloch")
    for gamma_m in [0.0] + np.logspace(np.log10(0.0001), np.log10(0.1), 9).tolist()
    for gamma_p in [0.0] + np.logspace(np.log10(0.0001), np.log10(0.1), 9).tolist()
]

# Formatting specification for experiment parameters
experiments_format = [ # Format of variables as tuples (name, type, required by grape, required by lut, required by rnn)
    ("t", int, True, True, True), # number timesteps (or segments)
    ("l", int, False, True, False), # LUT memory
    ("w", list, True, True, True), # reward weights
    ("Nqubits", int, True, True, True), # number of chains
    ("Nmeas", int, True, True, True), # number of measurements per timestep
    ("gammap", float, True, True, True), # sig^+ rate
    ("gammam", float, True, True, True), # sig^- rate
    ("rhot", str, True, True, True), # training state type
    ("rhoe", str, True, True, True), # evaluation state type
]

state_types = {
    "discrete": generate_random_discrete_state,
    "bloch": generate_random_bloch_state,
    "excited": generate_excited_state,
    "ground": generate_ground_state,
}


# Check if experiment parameters are in specified format
for params in experiments:
    assert len(params) == len(experiments_format), "Experiment parameters length mismatch."
    for i, (param, (name, expected_type, req_grape, req_lut, req_rnn)) in enumerate(zip(params, experiments_format)):
        assert type(name) == str, "Parameter name must be a string."
        assert type(req_grape) == bool, "Requirement flags must be boolean."
        assert type(req_lut) == bool, "Requirement flags must be boolean."
        assert type(req_rnn) == bool, "Requirement flags must be boolean."
        assert type(param) == expected_type, f"Parameter '{name}' at index {i} must be of type {expected_type.__name__}."
        if type(param) == list:
            for j, item in enumerate(param):
                assert type(item) == int and 0 <= item <= 9, f"Item at index {j} in parameter '{name}' must be of type int and between 0 and 9."

# Check if directory structure exists, create if not
for dir__ in [dir_, f"{dir_}/models", f"{dir_}/training_params", f"{dir_}/eval"]:
    os.makedirs(dir__, exist_ok=True)

# Generate parameter-based file paths
def generate_param_paths(dir_, params, model_type, s):
    assert model_type in ["grape", "lut", "rnn"], "Invalid model type."

    parts = []
    for (name, expected_type, req_grape, req_lut, req_rnn), param in zip(experiments_format, params):
        required = (model_type == "grape" and req_grape) or (model_type == "lut" and req_lut) or (model_type == "rnn" and req_rnn)
        if required:
            if expected_type == list:
                param_str = "".join(str(x) for x in param)
            else:
                param_str = str(param)
            parts.append(f"{name}={param_str}")

    pstr = f"{model_type}_" + "_".join(parts)
    pstr += f"_s={s}"
    
    return (
        f"{dir_}/models/{pstr}.json",
        f"{dir_}/training_params/{pstr}.json",
        f"{dir_}/eval/{pstr}.json",
    )

In [3]:
from concurrent.futures import ThreadPoolExecutor

def run_experiment(params):
    (
        num_time_steps,
        lut_memory,
        reward_weights,
        N_qubits,
        N_meas,
        gamma_p,
        gamma_m,
        training_state_type,
        evaluation_state_type
    ) = params
    
    print(f"Evaluating {params}")
    training_state = lambda key: state_types[training_state_type](key, N_qubits)
    evaluation_state = lambda key: state_types[evaluation_state_type](key, N_qubits)

    for s in tqdm(range(training_params["N_samples"])):
        system_params = init_fgrape_protocol(
            jax.random.PRNGKey(s + training_params["samples_offset"]),
            N_qubits,
            N_meas,
            gamma_p,
            gamma_m,
        )

        # Train LUT
        result = optimize_pulse(
            U_0=training_state,
            C_target=training_state,
            system_params=system_params,
            num_time_steps=num_time_steps,
            lut_depth=lut_memory,
            reward_weights=reward_weights,
            mode="lookup",
            goal="fidelity",
            max_iter=training_params["N_training_iterations"],
            convergence_threshold=training_params["convergence_threshold"],
            learning_rate=training_params["learning_rate"],
            evo_type="density",
            batch_size=training_params["batch_size"],
            eval_batch_size=1, # This evaluation is discarded
        )
        eval_result = evaluate_on_longer_time( # Evaluate on longer time and choose best LUT accordingly
            U_0 = evaluation_state,
            C_target = evaluation_state,
            system_params = system_params,
            optimized_trainable_parameters = result.optimized_trainable_parameters,
            num_time_steps = training_params["evaluation_time_steps"],
            evo_type = "density",
            goal = "fidelity",
            eval_batch_size = training_params["eval_batch_size"],
            mode = "lookup",
        )

        path1, path2, path3 = generate_param_paths(dir_, params, "lut", s)
        with open(path1, "w") as f:
            json.dump(FgResult_to_dict(result), f)

        with open(path2, "w") as f:
            json.dump(training_params, f)

        with open(path3, "w") as f:
            json.dump(FgResult_to_dict(eval_result), f)

# Run experiments in parallel
if N_parallel_threads > 1:
    with ThreadPoolExecutor(max_workers=N_parallel_threads) as executor:
        executor.map(run_experiment, experiments)
else:
    for params in experiments:
        run_experiment(params)

# Play a sound when done
os.system('say "done."')

Evaluating (3, 2, [1, 1, 1], 3, 2, 0.0, 0.001, 'bloch', 'bloch')
Evaluating (3, 2, [1, 1, 1], 3, 2, 0.0001, 0.001, 'bloch', 'bloch')
Evaluating (3, 2, [1, 1, 1], 3, 2, 0.0001333521432163324, 0.001, 'bloch', 'bloch')
Evaluating (3, 2, [1, 1, 1], 3, 2, 0.00017782794100389227, 0.001, 'bloch', 'bloch')
Evaluating (3, 2, [1, 1, 1], 3, 2, 0.00023713737056616554, 0.001, 'bloch', 'bloch')
Evaluating (3, 2, [1, 1, 1], 3, 2, 0.00031622776601683794, 0.001, 'bloch', 'bloch')
Evaluating (3, 2, [1, 1, 1], 3, 2, 0.00042169650342858224, 0.001, 'bloch', 'bloch')
Evaluating (3, 2, [1, 1, 1], 3, 2, 0.0005623413251903491, 0.001, 'bloch', 'bloch')












  0%|          | 0/5 [00:00<?, ?it/s]



















 20%|██        | 1/5 [04:03<16:15, 243.79s/it]
















 40%|████      | 2/5 [09:19<14:17, 285.82s/it]















 60%|██████    | 3/5 [15:39<10:57, 328.83s/it]






















 80%|████████  | 4/5 [23:54<06:34, 394.52s/it]




















100%|██████████| 5/5 [29:22<00:00, 352.58s/it]


Evaluating (3, 2, [1, 1, 1], 3, 2, 0.0007498942093324559, 0.001, 'bloch', 'bloch')






100%|██████████| 5/5 [32:38<00:00, 391.69s/it]


Evaluating (3, 2, [1, 1, 1], 3, 2, 0.001, 0.001, 'bloch', 'bloch')


...

Evaluating (3, 2, [1, 1, 1], 3, 2, 0.042169650342858224, 0.1, 'bloch', 'bloch')






















100%|██████████| 5/5 [56:23<00:00, 676.64s/it]


Evaluating (3, 2, [1, 1, 1], 3, 2, 0.1, 0.1, 'bloch', 'bloch')


100%|██████████| 5/5 [56:48<00:00, 681.78s/it]






100%|██████████| 5/5 [51:52<00:00, 622.46s/it]





100%|██████████| 5/5 [56:25<00:00, 677.15s/it]


100%|██████████| 5/5 [55:55<00:00, 671.03s/it]







 20%|██        | 1/5 [07:00<28:02, 420.56s/it]



 40%|████      | 2/5 [11:30<16:36, 332.05s/it]



100%|██████████| 5/5 [23:07<00:00, 277.43s/it]


0